# 🏭 실습 2: 베어링 진동 분석 & 오토인코더 이상 탐지

**소요시간: 2시간 | 난이도: ★★☆☆☆ (초급)**

---

## 🎯 학습 목표

- 베어링 진동 데이터의 특성과 결함 유형을 이해합니다
- 시간/주파수 도메인 피처 추출 기법을 마스터합니다
- 다양한 진동 분석 기법 (Envelope, Spectrogram, Wavelet)을 실습합니다
- **오토인코더(Autoencoder) 기반 이상 탐지를 구현합니다**

---

## 📋 사전 준비

- Google Colab 계정 (GPU 권장)
- Lab 1 실습 완료 (권장)
- 기본적인 신호 처리 개념

---

# Part 1: 베어링 진동 기초 이론 (20분)

## 1.1 베어링 구조와 결함 유형

베어링은 회전 기계의 핵심 부품으로, 결함 발생 시 특징적인 진동 패턴을 나타냅니다.

### 베어링 구성 요소
- **내륜 (Inner Race)**: 축에 고정, 축과 함께 회전
- **외륜 (Outer Race)**: 하우징에 고정, 정지 상태
- **전동체 (Rolling Element)**: 볼 또는 롤러
- **케이지 (Cage)**: 전동체 간격 유지

### 결함 주파수 (Fault Frequencies)

| 결함 위치 | 약어 | 특징 |
|-----------|------|------|
| 외륜 결함 | BPFO (Ball Pass Frequency Outer) | 가장 흔함, 일정한 충격 |
| 내륜 결함 | BPFI (Ball Pass Frequency Inner) | 하중대에서 변조 발생 |
| 볼/롤러 결함 | BSF (Ball Spin Frequency) | 2x BSF 성분 |
| 케이지 결함 | FTF (Fundamental Train Frequency) | 저주파 성분 |

### 결함 주파수 계산 공식

```
BPFO = (n/2) × fr × (1 - Bd/Pd × cos(θ))
BPFI = (n/2) × fr × (1 + Bd/Pd × cos(θ))
BSF = (Pd/2Bd) × fr × (1 - (Bd/Pd × cos(θ))²)
FTF = (1/2) × fr × (1 - Bd/Pd × cos(θ))

여기서:
n = 전동체 수
fr = 회전 주파수 (Hz)
Bd = 볼 직경
Pd = 피치 직경
θ = 접촉각
```

In [ ]:
!apt-get install -y fonts-nanum*
!rm -rf /root/.cache/matplotlib/* # 폰트 캐시 재설정
# 런타임 다시 시작

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
#
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
plt.rcParams['font.family'] = font_name

In [ ]:
# Step 1: 라이브러리 설치 및 임포트
!pip install -q torch pywavelets scipy

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, stats
from scipy.fft import fft, fftfreq, rfft, rfftfreq
from scipy.signal import hilbert, butter, filtfilt, stft
import pywt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import warnings
warnings.filterwarnings('ignore')

# 시각화 설정
plt.rcParams['figure.figsize'] = (14, 5)
plt.rcParams['font.size'] = 11

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'✅ 라이브러리 임포트 완료!')
print(f'🖥️ 사용 디바이스: {device}')

## 1.2 베어링 결함 주파수 계산기

In [ ]:
# Step 2: 베어링 결함 주파수 계산 클래스
class BearingFaultFrequency:
    '''
    베어링 결함 주파수 계산기
    '''
    def __init__(self, n_balls, ball_diameter, pitch_diameter, contact_angle=0):
        '''
        Parameters:
        -----------
        n_balls : int - 볼(전동체) 개수
        ball_diameter : float - 볼 직경 (mm)
        pitch_diameter : float - 피치 직경 (mm)
        contact_angle : float - 접촉각 (degrees)
        '''
        self.n = n_balls
        self.Bd = ball_diameter
        self.Pd = pitch_diameter
        self.theta = np.radians(contact_angle)

    def calculate_frequencies(self, rpm):
        '''
        주어진 RPM에서 결함 주파수 계산

        Returns: dict with BPFO, BPFI, BSF, FTF in Hz
        '''
        fr = rpm / 60  # 회전 주파수 (Hz)
        ratio = self.Bd / self.Pd * np.cos(self.theta)

        BPFO = (self.n / 2) * fr * (1 - ratio)  # 외륜 결함
        BPFI = (self.n / 2) * fr * (1 + ratio)  # 내륜 결함
        BSF = (self.Pd / (2 * self.Bd)) * fr * (1 - ratio**2)  # 볼 결함
        FTF = (1 / 2) * fr * (1 - ratio)  # 케이지 결함

        return {
            'shaft_freq': fr,
            'BPFO': BPFO,
            'BPFI': BPFI,
            'BSF': BSF,
            'FTF': FTF
        }

    def print_frequencies(self, rpm):
        '''결함 주파수 출력'''
        freqs = self.calculate_frequencies(rpm)
        print(f'\n🔧 베어링 결함 주파수 (RPM: {rpm})')
        print('=' * 40)
        print(f'  축 회전 주파수 (1X): {freqs["shaft_freq"]:.2f} Hz')
        print(f'  외륜 결함 (BPFO): {freqs["BPFO"]:.2f} Hz')
        print(f'  내륜 결함 (BPFI): {freqs["BPFI"]:.2f} Hz')
        print(f'  볼 결함 (BSF): {freqs["BSF"]:.2f} Hz')
        print(f'  케이지 결함 (FTF): {freqs["FTF"]:.2f} Hz')
        return freqs

# 예시: SKF 6205 베어링 (일반적인 모터용 베어링)
bearing_6205 = BearingFaultFrequency(
    n_balls=9,
    ball_diameter=7.94,  # mm
    pitch_diameter=38.5,  # mm
    contact_angle=0
)

# 1800 RPM에서 결함 주파수 계산
fault_freqs = bearing_6205.print_frequencies(rpm=1800)

---

# Part 2: 베어링 진동 데이터 생성 및 분석 (40분)

## 2.1 다양한 베어링 결함 신호 생성

In [ ]:
# Step 3: 베어링 진동 신호 생성 클래스
class BearingVibrationGenerator:
    '''
    베어링 진동 신호 생성기
    다양한 결함 유형의 진동 신호를 시뮬레이션합니다.
    '''
    def __init__(self, fs=12000, duration=1.0, rpm=1800):
        '''
        Parameters:
        -----------
        fs : int - 샘플링 주파수 (Hz)
        duration : float - 신호 길이 (초)
        rpm : int - 회전 속도
        '''
        self.fs = fs
        self.duration = duration
        self.rpm = rpm
        self.n_samples = int(fs * duration)
        self.t = np.linspace(0, duration, self.n_samples)

        # 베어링 파라미터 (SKF 6205 기준)
        self.bearing = BearingFaultFrequency(
            n_balls=9, ball_diameter=7.94,
            pitch_diameter=38.5, contact_angle=0
        )
        self.fault_freqs = self.bearing.calculate_frequencies(rpm)

    def generate_normal(self, noise_level=0.1):
        '''
        정상 상태 진동 신호 생성
        - 축 회전 주파수(1X)와 고조파
        - 작은 랜덤 노이즈
        '''
        fr = self.fault_freqs['shaft_freq']

        # 기본 회전 성분 (1X, 2X, 3X)
        sig = 0.5 * np.sin(2 * np.pi * fr * self.t)  # 1X
        sig += 0.2 * np.sin(2 * np.pi * 2 * fr * self.t)  # 2X
        sig += 0.1 * np.sin(2 * np.pi * 3 * fr * self.t)  # 3X

        # 노이즈 추가
        sig += noise_level * np.random.randn(self.n_samples)

        return sig, 'normal'

    def generate_outer_race_fault(self, severity=1.0, noise_level=0.1):
        '''
        외륜 결함 신호 생성 (BPFO)
        - 주기적인 충격 펄스
        - 고주파 링잉 (공진)
        '''
        fr = self.fault_freqs['shaft_freq']
        bpfo = self.fault_freqs['BPFO']

        # 기본 회전 성분
        sig = 0.3 * np.sin(2 * np.pi * fr * self.t)

        # BPFO 충격 펄스 생성
        impulse_period = 1.0 / bpfo
        n_impulses = int(self.duration / impulse_period)

        for i in range(n_impulses):
            impulse_time = i * impulse_period
            impulse_idx = int(impulse_time * self.fs)
            if impulse_idx < self.n_samples:
                # 감쇠 진동 (링잉) 추가
                t_impulse = self.t[impulse_idx:] - self.t[impulse_idx]
                ring_freq = 3000  # 공진 주파수
                damping = 500
                ringing = severity * np.exp(-damping * t_impulse) * np.sin(2 * np.pi * ring_freq * t_impulse)
                sig[impulse_idx:] += ringing[:len(sig) - impulse_idx]

        sig += noise_level * np.random.randn(self.n_samples)
        return sig, 'outer_race'

    def generate_inner_race_fault(self, severity=1.0, noise_level=0.1):
        '''
        내륜 결함 신호 생성 (BPFI)
        - BPFI 주파수의 충격
        - 축 회전 주파수로 진폭 변조 (하중대 효과)
        '''
        fr = self.fault_freqs['shaft_freq']
        bpfi = self.fault_freqs['BPFI']

        # 기본 회전 성분
        sig = 0.3 * np.sin(2 * np.pi * fr * self.t)

        # BPFI 충격 (1X로 변조됨)
        modulation = 0.5 + 0.5 * np.sin(2 * np.pi * fr * self.t)  # 1X 변조

        impulse_period = 1.0 / bpfi
        n_impulses = int(self.duration / impulse_period)

        for i in range(n_impulses):
            impulse_time = i * impulse_period
            impulse_idx = int(impulse_time * self.fs)
            if impulse_idx < self.n_samples:
                t_impulse = self.t[impulse_idx:] - self.t[impulse_idx]
                ring_freq = 3500
                damping = 600
                ringing = severity * modulation[impulse_idx] * np.exp(-damping * t_impulse) * np.sin(2 * np.pi * ring_freq * t_impulse)
                sig[impulse_idx:] += ringing[:len(sig) - impulse_idx]

        sig += noise_level * np.random.randn(self.n_samples)
        return sig, 'inner_race'

    def generate_ball_fault(self, severity=1.0, noise_level=0.1):
        '''
        볼(전동체) 결함 신호 생성 (BSF)
        - 2×BSF 주파수 성분 (볼이 내/외륜 양쪽에 접촉)
        - FTF로 변조
        '''
        fr = self.fault_freqs['shaft_freq']
        bsf = self.fault_freqs['BSF']
        ftf = self.fault_freqs['FTF']

        # 기본 회전 성분
        sig = 0.3 * np.sin(2 * np.pi * fr * self.t)

        # 2×BSF 성분 (케이지 주파수로 변조)
        modulation = 0.5 + 0.5 * np.sin(2 * np.pi * ftf * self.t)

        impulse_period = 1.0 / (2 * bsf)  # 2×BSF
        n_impulses = int(self.duration / impulse_period)

        for i in range(n_impulses):
            impulse_time = i * impulse_period
            impulse_idx = int(impulse_time * self.fs)
            if impulse_idx < self.n_samples:
                t_impulse = self.t[impulse_idx:] - self.t[impulse_idx]
                ring_freq = 2800
                damping = 550
                ringing = severity * 0.7 * modulation[impulse_idx] * np.exp(-damping * t_impulse) * np.sin(2 * np.pi * ring_freq * t_impulse)
                sig[impulse_idx:] += ringing[:len(sig) - impulse_idx]

        sig += noise_level * np.random.randn(self.n_samples)
        return sig, 'ball'

    def generate_cage_fault(self, severity=1.0, noise_level=0.1):
        '''
        케이지 결함 신호 생성 (FTF)
        - 저주파 FTF 성분
        - 불규칙한 진폭 변동
        '''
        fr = self.fault_freqs['shaft_freq']
        ftf = self.fault_freqs['FTF']

        # 기본 회전 성분
        sig = 0.3 * np.sin(2 * np.pi * fr * self.t)

        # FTF 저주파 성분
        sig += severity * 0.4 * np.sin(2 * np.pi * ftf * self.t)
        sig += severity * 0.2 * np.sin(2 * np.pi * 2 * ftf * self.t)

        # 불규칙한 변동 추가
        irregular = 0.15 * severity * np.sin(2 * np.pi * ftf * self.t) * np.random.randn(self.n_samples)
        sig += irregular

        sig += noise_level * np.random.randn(self.n_samples)
        return sig, 'cage'

    def generate_imbalance(self, severity=1.0, noise_level=0.1):
        '''
        불균형(Imbalance) 신호 생성
        - 1X 성분 증가
        '''
        fr = self.fault_freqs['shaft_freq']

        # 강한 1X 성분
        sig = (0.5 + severity * 0.8) * np.sin(2 * np.pi * fr * self.t)
        sig += 0.2 * np.sin(2 * np.pi * 2 * fr * self.t)
        sig += 0.1 * np.sin(2 * np.pi * 3 * fr * self.t)

        sig += noise_level * np.random.randn(self.n_samples)
        return sig, 'imbalance'

    def generate_misalignment(self, severity=1.0, noise_level=0.1):
        '''
        미스얼라인먼트(Misalignment) 신호 생성
        - 2X 성분 증가 (각도 미스얼라인먼트)
        - 축방향 진동 증가
        '''
        fr = self.fault_freqs['shaft_freq']

        # 1X와 강한 2X 성분
        sig = 0.5 * np.sin(2 * np.pi * fr * self.t)
        sig += (0.2 + severity * 0.6) * np.sin(2 * np.pi * 2 * fr * self.t)  # 강한 2X
        sig += 0.3 * np.sin(2 * np.pi * 3 * fr * self.t)

        sig += noise_level * np.random.randn(self.n_samples)
        return sig, 'misalignment'

print('✅ BearingVibrationGenerator 클래스 정의 완료!')

In [ ]:
# Step 4: 다양한 결함 신호 생성 및 시각화
np.random.seed(42)

generator = BearingVibrationGenerator(fs=12000, duration=0.5, rpm=1800)

# 각 상태별 신호 생성
signals = {
    'normal': generator.generate_normal(),
    'outer_race': generator.generate_outer_race_fault(severity=1.0),
    'inner_race': generator.generate_inner_race_fault(severity=1.0),
    'ball': generator.generate_ball_fault(severity=1.0),
    'cage': generator.generate_cage_fault(severity=1.0),
    'imbalance': generator.generate_imbalance(severity=1.0),
    'misalignment': generator.generate_misalignment(severity=1.0)
}

# 시간 영역 시각화
fig, axes = plt.subplots(4, 2, figsize=(16, 12))
axes = axes.flatten()

titles = {
    'normal': '정상 (Normal)',
    'outer_race': '외륜 결함 (BPFO)',
    'inner_race': '내륜 결함 (BPFI)',
    'ball': '볼 결함 (BSF)',
    'cage': '케이지 결함 (FTF)',
    'imbalance': '불균형 (Imbalance)',
    'misalignment': '미스얼라인먼트 (Misalignment)'
}

colors = ['green', 'red', 'orange', 'purple', 'brown', 'blue', 'cyan']

for idx, (name, (sig, _)) in enumerate(signals.items()):
    ax = axes[idx]
    ax.plot(generator.t[:2000], sig[:2000], color=colors[idx], alpha=0.8, linewidth=0.5)
    ax.set_title(titles[name], fontsize=12)
    ax.set_xlabel('시간 (초)')
    ax.set_ylabel('진폭')
    ax.grid(True, alpha=0.3)
    ax.set_xlim([0, generator.t[2000]])

# 마지막 빈 subplot 숨기기
axes[-1].axis('off')

plt.suptitle('베어링 결함 유형별 진동 신호 (시간 영역)', fontsize=14)
plt.tight_layout()
plt.show()

## 2.2 주파수 스펙트럼 분석 (FFT)

In [ ]:
# Step 5: FFT 스펙트럼 분석
def compute_spectrum(sig, fs):
    '''단측 주파수 스펙트럼 계산'''
    n = len(sig)
    freq = rfftfreq(n, 1/fs)
    fft_vals = np.abs(rfft(sig)) * 2 / n
    return freq, fft_vals

# 주파수 스펙트럼 시각화
fig, axes = plt.subplots(4, 2, figsize=(16, 12))
axes = axes.flatten()

fault_freqs = generator.fault_freqs

for idx, (name, (sig, _)) in enumerate(signals.items()):
    ax = axes[idx]
    freq, spectrum = compute_spectrum(sig, generator.fs)

    # 0~500Hz 범위만 표시
    mask = freq <= 500
    ax.plot(freq[mask], spectrum[mask], color=colors[idx], alpha=0.8)

    # 결함 주파수 표시
    if name != 'normal':
        ax.axvline(fault_freqs['shaft_freq'], color='gray', linestyle='--', alpha=0.5, label='1X')
        ax.axvline(fault_freqs['BPFO'], color='red', linestyle=':', alpha=0.7, label='BPFO')
        ax.axvline(fault_freqs['BPFI'], color='orange', linestyle=':', alpha=0.7, label='BPFI')

    ax.set_title(titles[name], fontsize=12)
    ax.set_xlabel('주파수 (Hz)')
    ax.set_ylabel('진폭')
    ax.grid(True, alpha=0.3)
    if name != 'normal':
        ax.legend(loc='upper right', fontsize=8)

axes[-1].axis('off')

plt.suptitle('베어링 결함 유형별 주파수 스펙트럼', fontsize=14)
plt.tight_layout()
plt.show()

## 2.3 엔벨로프 분석 (Envelope Analysis)

엔벨로프 분석은 베어링 결함 탐지에 가장 효과적인 방법 중 하나입니다.

1. **대역 통과 필터**: 공진 주파수 대역 추출
2. **힐버트 변환**: 순시 진폭(엔벨로프) 추출
3. **엔벨로프 스펙트럼**: 결함 주파수 검출

In [ ]:
# Step 6: 엔벨로프 분석 함수
def envelope_analysis(sig, fs, low_freq=2000, high_freq=5000):
    '''
    엔벨로프 분석 수행

    Parameters:
    -----------
    sig : array - 입력 신호
    fs : int - 샘플링 주파수
    low_freq, high_freq : float - 대역 통과 필터 범위

    Returns:
    --------
    envelope : array - 엔벨로프 신호
    env_freq : array - 엔벨로프 스펙트럼 주파수
    env_spectrum : array - 엔벨로프 스펙트럼
    '''
    # 1. 대역 통과 필터 설계
    nyquist = fs / 2
    low = low_freq / nyquist
    high = high_freq / nyquist

    # 주파수가 유효 범위인지 확인
    if high >= 1:
        high = 0.99

    b, a = butter(4, [low, high], btype='band')

    # 2. 대역 통과 필터 적용
    filtered = filtfilt(b, a, sig)

    # 3. 힐버트 변환으로 엔벨로프 추출
    analytic_signal = hilbert(filtered)
    envelope = np.abs(analytic_signal)

    # 4. DC 성분 제거
    envelope = envelope - np.mean(envelope)

    # 5. 엔벨로프 스펙트럼 계산
    env_freq, env_spectrum = compute_spectrum(envelope, fs)

    return envelope, env_freq, env_spectrum, filtered

print('✅ 엔벨로프 분석 함수 정의 완료!')

In [ ]:
# Step 7: 엔벨로프 분석 결과 시각화
fig, axes = plt.subplots(3, 3, figsize=(16, 12))

# 정상, 외륜결함, 내륜결함 비교
selected = ['normal', 'outer_race', 'inner_race']

for col, name in enumerate(selected):
    sig, _ = signals[name]
    envelope, env_freq, env_spectrum, filtered = envelope_analysis(sig, generator.fs)

    # 원본 신호
    axes[0, col].plot(generator.t[:1500], sig[:1500], 'b-', alpha=0.7, linewidth=0.5)
    axes[0, col].set_title(f'{titles[name]} - 원본 신호')
    axes[0, col].set_xlabel('시간 (초)')
    axes[0, col].set_ylabel('진폭')
    axes[0, col].grid(True, alpha=0.3)

    # 필터링 + 엔벨로프
    axes[1, col].plot(generator.t[:1500], filtered[:1500], 'gray', alpha=0.5, linewidth=0.5, label='필터링')
    axes[1, col].plot(generator.t[:1500], envelope[:1500], 'r-', linewidth=1, label='엔벨로프')
    axes[1, col].set_title(f'{titles[name]} - 엔벨로프')
    axes[1, col].set_xlabel('시간 (초)')
    axes[1, col].set_ylabel('진폭')
    axes[1, col].legend(fontsize=8)
    axes[1, col].grid(True, alpha=0.3)

    # 엔벨로프 스펙트럼
    mask = env_freq <= 300
    axes[2, col].plot(env_freq[mask], env_spectrum[mask], 'g-', alpha=0.8)
    axes[2, col].axvline(fault_freqs['BPFO'], color='red', linestyle='--', alpha=0.7, label=f'BPFO={fault_freqs["BPFO"]:.1f}Hz')
    axes[2, col].axvline(fault_freqs['BPFI'], color='orange', linestyle='--', alpha=0.7, label=f'BPFI={fault_freqs["BPFI"]:.1f}Hz')
    axes[2, col].set_title(f'{titles[name]} - 엔벨로프 스펙트럼')
    axes[2, col].set_xlabel('주파수 (Hz)')
    axes[2, col].set_ylabel('진폭')
    axes[2, col].legend(fontsize=8)
    axes[2, col].grid(True, alpha=0.3)

plt.suptitle('엔벨로프 분석 비교', fontsize=14)
plt.tight_layout()
plt.show()

## 2.4 스펙트로그램 (STFT) 분석

시간에 따른 주파수 변화를 관찰할 수 있습니다.

In [ ]:
# Step 8: 스펙트로그램 분석
def compute_spectrogram(sig, fs, nperseg=256, noverlap=128):
    '''STFT 기반 스펙트로그램 계산'''
    f, t, Zxx = stft(sig, fs, nperseg=nperseg, noverlap=noverlap)
    return f, t, np.abs(Zxx)

# 스펙트로그램 시각화
fig, axes = plt.subplots(2, 3, figsize=(16, 8))

selected = ['normal', 'outer_race', 'inner_race', 'ball', 'imbalance', 'misalignment']

for idx, name in enumerate(selected):
    row, col = idx // 3, idx % 3
    sig, _ = signals[name]
    f, t, Sxx = compute_spectrogram(sig, generator.fs)

    # 0~1000Hz 범위만 표시
    f_mask = f <= 1000

    im = axes[row, col].pcolormesh(t, f[f_mask], 20*np.log10(Sxx[f_mask] + 1e-10),
                                    shading='gouraud', cmap='jet')
    axes[row, col].set_title(titles[name])
    axes[row, col].set_xlabel('시간 (초)')
    axes[row, col].set_ylabel('주파수 (Hz)')
    plt.colorbar(im, ax=axes[row, col], label='dB')

plt.suptitle('베어링 결함 유형별 스펙트로그램', fontsize=14)
plt.tight_layout()
plt.show()

## 2.5 웨이블릿 분석 (Wavelet Transform)

웨이블릿 변환은 비정상 신호 분석에 효과적입니다.

In [ ]:
# Step 9: 웨이블릿 분석
def wavelet_analysis(sig, wavelet='morl', scales=None, fs=12000):
    '''
    연속 웨이블릿 변환 (CWT) 수행
    '''
    if scales is None:
        # 주파수 범위에 맞는 스케일 계산
        scales = np.arange(1, 128)

    coefficients, frequencies = pywt.cwt(sig, scales, wavelet, sampling_period=1/fs)
    return coefficients, frequencies, scales

# 웨이블릿 스칼로그램 시각화
fig, axes = plt.subplots(2, 3, figsize=(16, 8))

for idx, name in enumerate(selected):
    row, col = idx // 3, idx % 3
    sig, _ = signals[name]

    # 계산 시간 단축을 위해 일부 샘플만 사용
    sig_short = sig[:3000]
    t_short = generator.t[:3000]

    coeffs, freqs, scales = wavelet_analysis(sig_short, fs=generator.fs)

    # 0~500Hz 범위만 표시
    f_mask = freqs <= 500

    im = axes[row, col].pcolormesh(t_short, freqs[f_mask], np.abs(coeffs[f_mask]),
                                    shading='gouraud', cmap='jet')
    axes[row, col].set_title(titles[name])
    axes[row, col].set_xlabel('시간 (초)')
    axes[row, col].set_ylabel('주파수 (Hz)')
    plt.colorbar(im, ax=axes[row, col])

plt.suptitle('베어링 결함 유형별 웨이블릿 스칼로그램 (Morlet)', fontsize=14)
plt.tight_layout()
plt.show()

## 2.6 통계적 피처 추출

In [ ]:
# Step 10: 종합 피처 추출 함수
def extract_features(sig, fs):
    '''
    시간/주파수 도메인 피처 추출
    '''
    features = {}

    # === 시간 도메인 피처 ===
    features['mean'] = np.mean(sig)
    features['std'] = np.std(sig)
    features['rms'] = np.sqrt(np.mean(sig**2))
    features['peak'] = np.max(np.abs(sig))
    features['peak_to_peak'] = np.max(sig) - np.min(sig)
    features['crest_factor'] = features['peak'] / features['rms']
    features['shape_factor'] = features['rms'] / np.mean(np.abs(sig))
    features['impulse_factor'] = features['peak'] / np.mean(np.abs(sig))
    features['clearance_factor'] = features['peak'] / (np.mean(np.sqrt(np.abs(sig)))**2)
    features['kurtosis'] = stats.kurtosis(sig)
    features['skewness'] = stats.skew(sig)

    # === 주파수 도메인 피처 ===
    freq, spectrum = compute_spectrum(sig, fs)

    # 주파수 중심
    features['spectral_centroid'] = np.sum(freq * spectrum) / np.sum(spectrum)

    # 주파수 분산
    features['spectral_spread'] = np.sqrt(np.sum(((freq - features['spectral_centroid'])**2) * spectrum) / np.sum(spectrum))

    # 스펙트럼 첨도
    features['spectral_kurtosis'] = stats.kurtosis(spectrum)

    # 지배 주파수
    features['dominant_freq'] = freq[np.argmax(spectrum)]

    # 주파수 대역별 에너지
    for band_name, (f_low, f_high) in [('low', (0, 100)), ('mid', (100, 500)), ('high', (500, 2000))]:
        mask = (freq >= f_low) & (freq < f_high)
        features[f'energy_{band_name}'] = np.sum(spectrum[mask]**2)

    # === 엔벨로프 피처 ===
    try:
        envelope, env_freq, env_spectrum, _ = envelope_analysis(sig, fs)
        features['envelope_rms'] = np.sqrt(np.mean(envelope**2))
        features['envelope_peak'] = np.max(np.abs(envelope))
        features['envelope_kurtosis'] = stats.kurtosis(envelope)
    except:
        features['envelope_rms'] = 0
        features['envelope_peak'] = 0
        features['envelope_kurtosis'] = 0

    return features

# 모든 신호에 대해 피처 추출
feature_list = []
for name, (sig, label) in signals.items():
    features = extract_features(sig, generator.fs)
    features['label'] = label
    feature_list.append(features)

feature_df = pd.DataFrame(feature_list)
print('✅ 피처 추출 완료!\n')
print(feature_df.round(3))

---

# Part 3: 오토인코더 기반 이상 탐지 (50분)

## 3.1 오토인코더 개요

**오토인코더(Autoencoder)**는 입력을 압축(인코딩)했다가 복원(디코딩)하는 신경망입니다.

### 이상 탐지 원리
1. **정상 데이터로만 학습**: 정상 패턴을 학습
2. **복원 오차 계산**: 입력과 출력의 차이(MSE)
3. **이상 판정**: 복원 오차가 임계값을 초과하면 이상

```
입력(x) → [인코더] → 잠재 공간(z) → [디코더] → 출력(x')
                         ↓
                   차원 축소된 표현
                         ↓
              복원 오차 = ||x - x'||²
```

In [ ]:
# Step 11: 학습 데이터 생성
np.random.seed(42)

# 데이터 생성기
generator = BearingVibrationGenerator(fs=12000, duration=0.5, rpm=1800)

# 정상 데이터 생성 (학습용)
n_normal_train = 500
n_normal_val = 100

normal_train_signals = []
for i in range(n_normal_train):
    sig, _ = generator.generate_normal(noise_level=np.random.uniform(0.05, 0.15))
    normal_train_signals.append(sig)

normal_val_signals = []
for i in range(n_normal_val):
    sig, _ = generator.generate_normal(noise_level=np.random.uniform(0.05, 0.15))
    normal_val_signals.append(sig)

# 이상 데이터 생성 (테스트용)
n_anomaly = 50
anomaly_signals = []
anomaly_labels = []

fault_generators = [
    ('outer_race', generator.generate_outer_race_fault),
    ('inner_race', generator.generate_inner_race_fault),
    ('ball', generator.generate_ball_fault),
    ('cage', generator.generate_cage_fault),
    ('imbalance', generator.generate_imbalance),
    ('misalignment', generator.generate_misalignment)
]

for fault_name, fault_func in fault_generators:
    for i in range(n_anomaly // len(fault_generators)):
        severity = np.random.uniform(0.5, 1.5)
        sig, _ = fault_func(severity=severity)
        anomaly_signals.append(sig)
        anomaly_labels.append(fault_name)

print(f'✅ 데이터 생성 완료!')
print(f'   정상 학습 데이터: {len(normal_train_signals)}개')
print(f'   정상 검증 데이터: {len(normal_val_signals)}개')
print(f'   이상 테스트 데이터: {len(anomaly_signals)}개')

In [ ]:
# Step 12: 데이터 전처리 (피처 추출)
def signals_to_features(signals_list, fs=12000):
    '''신호 리스트를 피처 행렬로 변환'''
    features = []
    for sig in signals_list:
        feat = extract_features(sig, fs)
        # label 키 제거
        if 'label' in feat:
            del feat['label']
        features.append(list(feat.values()))
    return np.array(features)

# 피처 추출
print('피처 추출 중...')
X_train = signals_to_features(normal_train_signals)
X_val = signals_to_features(normal_val_signals)
X_anomaly = signals_to_features(anomaly_signals)

print(f'✅ 피처 추출 완료!')
print(f'   X_train shape: {X_train.shape}')
print(f'   X_val shape: {X_val.shape}')
print(f'   X_anomaly shape: {X_anomaly.shape}')

In [ ]:
# Step 13: 데이터 정규화
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_anomaly_scaled = scaler.transform(X_anomaly)

# PyTorch 텐서로 변환
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_val_tensor = torch.FloatTensor(X_val_scaled)
X_anomaly_tensor = torch.FloatTensor(X_anomaly_scaled)

# DataLoader 생성
batch_size = 32
train_dataset = TensorDataset(X_train_tensor, X_train_tensor)  # 입력 = 타겟
val_dataset = TensorDataset(X_val_tensor, X_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f'✅ 데이터 전처리 완료!')
print(f'   입력 차원: {X_train_scaled.shape[1]}')

## 3.2 오토인코더 모델 정의

In [ ]:
# Step 14: 오토인코더 모델 정의
class Autoencoder(nn.Module):
    '''
    완전 연결 오토인코더
    '''
    def __init__(self, input_dim, encoding_dim=8):
        super(Autoencoder, self).__init__()

        # 인코더
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),

            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),

            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),

            nn.Linear(16, encoding_dim),
            nn.ReLU()
        )

        # 디코더
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),

            nn.Linear(16, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.2),

            nn.Linear(32, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),

            nn.Linear(64, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def encode(self, x):
        return self.encoder(x)

# 모델 초기화
input_dim = X_train_scaled.shape[1]
encoding_dim = 8

model = Autoencoder(input_dim, encoding_dim).to(device)
print(f'✅ 오토인코더 모델 생성!')
print(f'   입력 차원: {input_dim}')
print(f'   잠재 공간 차원: {encoding_dim}')
print(f'\n모델 구조:')
print(model)

## 3.3 모델 학습

In [ ]:
# Step 15: 학습 설정
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10)

# 학습 기록
history = {'train_loss': [], 'val_loss': []}

# 조기 종료 설정
best_val_loss = float('inf')
patience = 20
patience_counter = 0

print('🚀 오토인코더 학습 시작!\n')

In [ ]:
# Step 16: 학습 루프
n_epochs = 100

for epoch in range(n_epochs):
    # 학습
    model.train()
    train_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_x)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    history['train_loss'].append(train_loss)

    # 검증
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x = batch_x.to(device)
            outputs = model(batch_x)
            loss = criterion(outputs, batch_x)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    history['val_loss'].append(val_loss)

    # 학습률 스케줄러
    scheduler.step(val_loss)

    # 조기 종료 체크
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # 최적 모델 저장
        torch.save(model.state_dict(), 'best_autoencoder.pth')
    else:
        patience_counter += 1

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}] Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}')

    if patience_counter >= patience:
        print(f'\n조기 종료! (Epoch {epoch+1})')
        break

# 최적 모델 로드
model.load_state_dict(torch.load('best_autoencoder.pth'))
print(f'\n✅ 학습 완료! 최적 검증 손실: {best_val_loss:.6f}')

In [ ]:
# 학습 과정 시각화
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss', color='blue')
plt.plot(history['val_loss'], label='Validation Loss', color='red')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('오토인코더 학습 과정')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 3.4 이상 탐지 및 평가

In [ ]:
# Step 17: 복원 오차 계산
def compute_reconstruction_error(model, data_tensor, device):
    '''복원 오차 계산'''
    model.eval()
    with torch.no_grad():
        data = data_tensor.to(device)
        reconstructed = model(data)
        errors = torch.mean((data - reconstructed) ** 2, dim=1)
        return errors.cpu().numpy()

# 각 데이터셋의 복원 오차 계산
train_errors = compute_reconstruction_error(model, X_train_tensor, device)
val_errors = compute_reconstruction_error(model, X_val_tensor, device)
anomaly_errors = compute_reconstruction_error(model, X_anomaly_tensor, device)

print('📊 복원 오차 통계:')
print(f'   정상 (학습): 평균 = {train_errors.mean():.6f}, 표준편차 = {train_errors.std():.6f}')
print(f'   정상 (검증): 평균 = {val_errors.mean():.6f}, 표준편차 = {val_errors.std():.6f}')
print(f'   이상: 평균 = {anomaly_errors.mean():.6f}, 표준편차 = {anomaly_errors.std():.6f}')

In [ ]:
# Step 18: 복원 오차 분포 시각화
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 히스토그램
axes[0].hist(train_errors, bins=50, alpha=0.7, label='정상 (학습)', color='green', density=True)
axes[0].hist(anomaly_errors, bins=50, alpha=0.7, label='이상', color='red', density=True)
axes[0].set_xlabel('복원 오차 (MSE)')
axes[0].set_ylabel('밀도')
axes[0].set_title('복원 오차 분포')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 임계값 결정 (정상 데이터 기준 99 백분위수)
threshold = np.percentile(train_errors, 99)
axes[0].axvline(threshold, color='black', linestyle='--', linewidth=2, label=f'임계값 = {threshold:.4f}')
axes[0].legend()

# 박스플롯
all_errors = [train_errors, val_errors, anomaly_errors]
axes[1].boxplot(all_errors, labels=['정상 (학습)', '정상 (검증)', '이상'])
axes[1].axhline(threshold, color='red', linestyle='--', linewidth=2, label=f'임계값')
axes[1].set_ylabel('복원 오차 (MSE)')
axes[1].set_title('복원 오차 박스플롯')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f'\n🎯 설정된 임계값: {threshold:.6f}')

In [ ]:
# Step 19: 이상 탐지 성능 평가
from sklearn.metrics import precision_recall_curve, roc_curve, auc, classification_report, confusion_matrix

# 라벨 생성
y_true = np.concatenate([np.zeros(len(val_errors)), np.ones(len(anomaly_errors))])
y_scores = np.concatenate([val_errors, anomaly_errors])

# 임계값으로 예측
y_pred = (y_scores > threshold).astype(int)

# ROC 곡선
fpr, tpr, _ = roc_curve(y_true, y_scores)
roc_auc = auc(fpr, tpr)

# PR 곡선
precision, recall, _ = precision_recall_curve(y_true, y_scores)
pr_auc = auc(recall, precision)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ROC 곡선
axes[0].plot(fpr, tpr, color='blue', linewidth=2, label=f'ROC (AUC = {roc_auc:.3f})')
axes[0].plot([0, 1], [0, 1], color='gray', linestyle='--')
axes[0].set_xlabel('False Positive Rate')
axes[0].set_ylabel('True Positive Rate')
axes[0].set_title('ROC 곡선')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# PR 곡선
axes[1].plot(recall, precision, color='green', linewidth=2, label=f'PR (AUC = {pr_auc:.3f})')
axes[1].set_xlabel('Recall')
axes[1].set_ylabel('Precision')
axes[1].set_title('Precision-Recall 곡선')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 분류 리포트
print('\n📊 분류 성능 리포트:')
print(classification_report(y_true, y_pred, target_names=['정상', '이상']))

In [ ]:
# Step 20: 혼동행렬 시각화
import seaborn as sns

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['정상 (예측)', '이상 (예측)'],
            yticklabels=['정상 (실제)', '이상 (실제)'],
            annot_kws={'size': 16})
plt.title('오토인코더 이상 탐지 혼동행렬')
plt.show()

## 3.5 결함 유형별 분석

In [ ]:
# Step 21: 결함 유형별 복원 오차 분석
# 결함 유형별로 데이터 분리
fault_errors = {}
for fault_name in set(anomaly_labels):
    indices = [i for i, label in enumerate(anomaly_labels) if label == fault_name]
    fault_errors[fault_name] = anomaly_errors[indices]

# 시각화
fig, ax = plt.subplots(figsize=(12, 6))

fault_names = list(fault_errors.keys())
fault_data = [fault_errors[name] for name in fault_names]

# 박스플롯
bp = ax.boxplot([train_errors] + fault_data,
                labels=['정상'] + fault_names,
                patch_artist=True)

colors = ['green'] + ['red', 'orange', 'purple', 'brown', 'blue', 'cyan'][:len(fault_names)]
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.axhline(threshold, color='black', linestyle='--', linewidth=2, label=f'임계값 = {threshold:.4f}')
ax.set_ylabel('복원 오차 (MSE)')
ax.set_title('결함 유형별 복원 오차 분포')
ax.legend()
ax.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 결함 유형별 탐지율
print('\n📊 결함 유형별 탐지율:')
for fault_name in fault_names:
    errors = fault_errors[fault_name]
    detection_rate = np.mean(errors > threshold) * 100
    print(f'   {fault_name}: {detection_rate:.1f}% ({np.sum(errors > threshold)}/{len(errors)})')

## 3.6 잠재 공간 시각화

In [ ]:
# Step 22: 잠재 공간 시각화 (t-SNE)
from sklearn.manifold import TSNE

# 인코딩된 표현 추출
model.eval()
with torch.no_grad():
    train_encoded = model.encode(X_train_tensor.to(device)).cpu().numpy()
    anomaly_encoded = model.encode(X_anomaly_tensor.to(device)).cpu().numpy()

# 결합
all_encoded = np.vstack([train_encoded[:100], anomaly_encoded])  # 시각화를 위해 일부만 사용
all_labels = ['정상'] * 100 + anomaly_labels

# t-SNE 적용
print('t-SNE 계산 중...')
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
encoded_2d = tsne.fit_transform(all_encoded)

# 시각화
plt.figure(figsize=(12, 8))

unique_labels = list(set(all_labels))
colors = {'정상': 'green', 'outer_race': 'red', 'inner_race': 'orange',
          'ball': 'purple', 'cage': 'brown', 'imbalance': 'blue', 'misalignment': 'cyan'}

for label in unique_labels:
    mask = [l == label for l in all_labels]
    plt.scatter(encoded_2d[mask, 0], encoded_2d[mask, 1],
                c=colors.get(label, 'gray'), label=label, alpha=0.7, s=50)

plt.xlabel('t-SNE 차원 1')
plt.ylabel('t-SNE 차원 2')
plt.title('오토인코더 잠재 공간 시각화 (t-SNE)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

# 📌 핵심 정리

## 베어링 진동 분석
1. **결함 주파수**: BPFO, BPFI, BSF, FTF는 베어링 기하학으로 계산 가능
2. **엔벨로프 분석**: 베어링 결함 탐지에 가장 효과적인 방법
3. **스펙트로그램/웨이블릿**: 시간에 따른 주파수 변화 관찰

## 오토인코더 이상 탐지
4. **비지도 학습**: 정상 데이터만으로 학습 가능
5. **복원 오차**: 이상 데이터는 높은 복원 오차를 가짐
6. **임계값 설정**: 정상 데이터의 백분위수 기반 결정

---

# 🚀 도전 과제

1. **VAE (Variational Autoencoder)** 구현하여 성능 비교
2. **1D CNN Autoencoder** 구현 (원시 신호 직접 입력)
3. **LSTM Autoencoder** 구현 (시계열 특성 활용)
4. 실제 **CWRU Bearing Dataset** 다운로드하여 적용

In [ ]:
# 모델 저장
torch.save({
    'model_state_dict': model.state_dict(),
    'scaler': scaler,
    'threshold': threshold,
    'input_dim': input_dim,
    'encoding_dim': encoding_dim
}, 'bearing_anomaly_detector.pth')

print('✅ 모델 저장 완료: bearing_anomaly_detector.pth')
print('\n🎉 실습 2 완료! 다음 실습으로 넘어가세요.')

---

**➡️ 다음 실습: 배포/연동 & 미니 PoC**